You have to add a file `keys.py` containing your keys:

``` python
API_KEY = 'API_KEY'
API_SECRET = 'API_SECRET'
ACCES_TOKEN = 'ACCES_TOKEN'
ACCES_TOKEN_SECRET = 'ACCES_TOKEN_SECRET'
```

In [ ]:
import tweepy
import keys as keys
import os
import urllib
import time
from progressbar import progressbar

In [ ]:
twitter_auth_keys = {
    "consumer_key": keys.API_KEY,
    "consumer_secret": keys.API_SECRET,
    "access_token": keys.ACCES_TOKEN,
    "access_token_secret": keys.ACCES_TOKEN_SECRET
}
auth = tweepy.OAuthHandler(
    twitter_auth_keys['consumer_key'],
    twitter_auth_keys['consumer_secret']
)
auth.set_access_token(
    twitter_auth_keys['access_token'],
    twitter_auth_keys['access_token_secret']
)
api = tweepy.API(auth)

In [ ]:
valids_image_types = ('jpg', 'png')
def get_image_url_from_tweet(tweet):    
    valids = []
    if 'media' in tweet.entities:
        for media in tweet.entities['media']:
            if media['media_url'].endswith(valids_image_types):
                if 'tweet_video_thumb' in media['media_url']:
                    pass
                else:
                    tweet_data = {'tweet_id': tweet.id,
                                 'media_url': media['media_url']}
                    valids.append(tweet_data)
    return valids

In [ ]:
user_name = 'FlagsMashupBot'
data_path = './data/{}/raw/'.format(user_name)
os.makedirs(data_path, exist_ok=True)

In [ ]:
# tweets = api.user_timeline(screen_name=user_name, count=100, )

#### get images urls

In [ ]:
current_media = os.listdir(data_path)
oldest_id = min([int(m.split('.')[0]) for m in current_media]) if len(current_media)>0 else None

In [ ]:
tweet_count = 0
max_tweets = 5000
tweets_per_query = 100
# oldest_id = min([int(m.split('.')[0]) for m in current_media]) if len(current_media)>0 else None
oldest_id = None

images_urls = []

api_kwargs = {
    'screen_name': user_name,
    'tweet_mode': 'extended',
    'count': tweets_per_query}

while tweet_count < max_tweets:
    print(oldest_id)
    if not oldest_id:
        tweets = api.user_timeline(**api_kwargs)
    else:
        tweets = api.user_timeline(max_id=oldest_id-1, **api_kwargs)
    for tweet in tweets:
        images_urls += get_image_url_from_tweet(tweet)
    oldest_id = min([int(t.id) for t in tweets])
    tweet_count +=len(tweets)


    
        

In [ ]:
# tweets = api.user_timeline(since_id=100, **api_kwargs)
# [t.id for t in tweets]

#### download images from urls

In [ ]:
for media in progressbar(images_urls):    
    file_ext = media['media_url'].split('.')[-1]
    file_name = '{}.{}'.format(media['tweet_id'], file_ext)
    file_path = '{}/{}'.format(data_path, file_name)
#     urllib.request.urlretrieve(media['media_url'], file_path)
#     ----
#     r = requests.get(media['media_url'])
#     open(file_path, 'wb').write(r.content)
    current_media = os.listdir(data_path)
    if file_name not in current_media:
        os.system('wget -O {} {}'.format(file_path, media['media_url']))
#         time.sleep(0.1)
    